In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Fake_Real_Data.csv")
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df.shape

(9900, 2)

In [6]:
df.label.value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [7]:
df["label_num"] = df.label.map({"Fake":0, "Real":1})

In [9]:
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [11]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [14]:
doc = nlp("Top Trump Surrogate BRUTALLY Stabs Him In The Back")
doc.vector.shape

(300,)

In [15]:
df["vector"] = df.Text.apply(lambda x: nlp(x).vector)

In [17]:
df.head()

,Text,label,label_num,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."


In [21]:
from sklearn.model_selection import train_test_split

In [26]:
x_train,x_test, y_train, y_test = train_test_split(df.vector.values,
                                                     df.label_num,
                                                    test_size = 0.2,
                                                    random_state = 2022)


In [27]:
x_train

array([array([-2.26311374e+00,  3.07298154e-01, -1.33053243e+00,  8.23658347e-01,
               4.81299686e+00, -6.86131325e-03,  3.35425436e-01,  3.76113081e+00,
               5.39720833e-01, -1.42519677e+00,  4.56928253e+00,  1.55305433e+00,
              -3.53332734e+00,  6.87306941e-01, -5.81286728e-01,  2.71541500e+00,
               1.39683473e+00,  9.30782378e-01, -8.92450333e-01, -1.38076222e+00,
               1.36910903e+00, -1.13511407e+00, -9.53715205e-01,  8.05722892e-01,
               6.48910582e-01, -1.65908492e+00, -2.64194083e+00, -5.77975094e-01,
              -1.42578706e-02,  2.33008236e-01,  2.95631811e-02,  2.99089730e-01,
              -4.90643919e-01, -1.33279932e+00, -2.74737763e+00, -9.77295876e-01,
              -5.31269848e-01,  1.10031247e+00,  4.24643457e-01, -3.41824055e-01,
               2.72467971e-01,  8.62076208e-02, -2.81822890e-01, -2.08477139e-01,
              -1.27764702e+00,  8.22918952e-01,  7.95027316e-01, -1.89551938e+00,
              -6

In [28]:
import numpy as np

x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(x_train_2d)
scaled_test_embed = scaler.transform(x_test_2d)


clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [34]:
from sklearn.metrics import classification_report
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test ,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1024
           1       0.94      0.95      0.94       956

    accuracy                           0.94      1980
   macro avg       0.94      0.94      0.94      1980
weighted avg       0.94      0.94      0.94      1980



In [35]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')
clf.fit(scaled_train_embed, y_train)
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test ,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1024
           1       0.99      0.99      0.99       956

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980



In [41]:
import spacy
df = pd.read_json("news_dataset.json")
df.head()

,text,category
0,Watching Schrödinger's Cat Die University of C...,SCIENCE
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,SCIENCE
2,Entrepreneurs Today Don't Need a Big Budget to...,BUSINESS
3,These Roads Could Recharge Your Electric Car A...,BUSINESS
4,Civilian 'Guard' Fires Gun While 'Protecting' ...,CRIME


In [43]:
df.shape

(12695, 2)

In [44]:
df.category.value_counts()

category
BUSINESS    4254
SPORTS      4167
CRIME       2893
SCIENCE     1381
Name: count, dtype: int64

In [45]:
minimum_sample = 1381

df_Business = df[df.category == "BUSINESS" ].sample(minimum_sample, random_state = 2022)
df_Sports = df[df.category == "SPORTS" ].sample(minimum_sample, random_state = 2022)
df_Crime = df[df.category == "CRIME" ].sample(minimum_sample, random_state = 2022)
df_Science = df[df.category == "SCIENCE" ].sample(minimum_sample, random_state = 2022)

In [48]:
df_new = pd.concat([df_Business, df_Sports, df_Crime, df_Science], axis = 0)
df_new.head()

,text,category
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS
2912,From the Other Side; an Honest Review from Emp...,BUSINESS
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS
502,How to Market Your Business While Traveling th...,BUSINESS
5279,How to Leverage Intuition in Decision-making I...,BUSINESS


In [50]:
df_new.category.value_counts()

category
BUSINESS    1381
SPORTS      1381
CRIME       1381
SCIENCE     1381
Name: count, dtype: int64

In [57]:
df_new["label_num"] = df_new.category.map({"BUSINESS":0, "SPORTS":1, "CRIME":2 , "SCIENCE":3})
df_new.head()

,text,category,label_num
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0
502,How to Market Your Business While Traveling th...,BUSINESS,0
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0


In [62]:
# Remove stop words 

def preprocess(text):
    doc = nlp(text)
    fil_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        fil_tokens.append(token.lemma_)
    return " ".join(fil_tokens)
    

In [63]:
df_new["preprocessed_text"] = df_new.text.apply(preprocess)

In [64]:
df_new.head()

,text,category,label_num,preprocessed_text
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0,GCC Business Leaders remain Confident face Reg...
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0,Honest Review employee wake morning love impor...
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0,Mike McDerment ceo FreshBooks talk give build ...
502,How to Market Your Business While Traveling th...,BUSINESS,0,market business travel World recently amazing ...
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0,leverage intuition decision making feel safe r...


In [66]:
df_new["vector"] = df_new.text.apply(lambda x: nlp(x).vector)

In [68]:
df_new.head()

,text,category,label_num,preprocessed_text,vector
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0,GCC Business Leaders remain Confident face Reg...,"[-2.8035183, -0.28117472, -2.0023017, 1.190918..."
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0,Honest Review employee wake morning love impor...,"[-1.3427333, 2.0853205, -3.1617253, 0.24224624..."
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0,Mike McDerment ceo FreshBooks talk give build ...,"[-1.422131, 1.1645982, -2.6296644, -0.29699433..."
502,How to Market Your Business While Traveling th...,BUSINESS,0,market business travel World recently amazing ...,"[-1.2492145, 1.084828, -2.5592022, -1.1566604,..."
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0,leverage intuition decision making feel safe r...,"[-0.7370661, 1.8736029, -3.4107106, -0.2069451..."


In [71]:
import numpy as np
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(df_new.vector, 
                                                   df_new.label_num,
                                                   test_size = 0.2,
                                                   random_state = 2022)

In [72]:
x_train.shape

(4419,)

In [73]:
x_test.shape

(1105,)

In [89]:
x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [90]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(x_train_2d)
scaled_test_embed = scaler.transform(x_test_2d)

In [100]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clf = DecisionTreeClassifier()
clf.fit(x_train_2d,y_train)


DecisionTreeClassifier()

In [101]:
y_pred = clf.predict(x_test_2d)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.48      0.48       269
           1       0.51      0.49      0.50       291
           2       0.52      0.57      0.54       245
           3       0.51      0.48      0.49       300

    accuracy                           0.50      1105
   macro avg       0.50      0.51      0.50      1105
weighted avg       0.50      0.50      0.50      1105



In [94]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(scaled_train_embed,y_train)
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.69      0.56       269
           1       0.80      0.49      0.61       291
           2       0.59      0.73      0.65       245
           3       0.66      0.49      0.56       300

    accuracy                           0.59      1105
   macro avg       0.63      0.60      0.60      1105
weighted avg       0.63      0.59      0.59      1105



In [96]:
from  sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(scaled_train_embed,y_train)
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.71      0.66       269
           1       0.70      0.66      0.68       291
           2       0.62      0.80      0.70       245
           3       0.79      0.51      0.62       300

    accuracy                           0.67      1105
   macro avg       0.68      0.67      0.66      1105
weighted avg       0.68      0.67      0.66      1105



In [97]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(scaled_train_embed,y_train)
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.71      0.69       269
           1       0.76      0.70      0.73       291
           2       0.74      0.80      0.77       245
           3       0.75      0.71      0.73       300

    accuracy                           0.73      1105
   macro avg       0.73      0.73      0.73      1105
weighted avg       0.73      0.73      0.73      1105



In [99]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(scaled_train_embed,y_train)
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.75      0.80      0.77       269
           1       0.84      0.76      0.80       291
           2       0.77      0.83      0.80       245
           3       0.80      0.78      0.79       300

    accuracy                           0.79      1105
   macro avg       0.79      0.79      0.79      1105
weighted avg       0.79      0.79      0.79      1105



In [102]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)